In [1]:
import boto3
import json
import base64


def understand_image(image_path, prompt):
    # Bedrock Runtime 클라이언트 생성 (원하는 리전에 맞게 설정)
    client = boto3.client(
        "bedrock-runtime"
    )

    # 사용할 모델 ID 지정 (Nova Lite 비전 모델)
    MODEL_ID = "amazon.nova-lite-v1:0"

    # 이미지 파일 열기 및 Base64 인코딩
    with open(image_path, "rb") as image_file:
        binary_data = image_file.read()  # 이미지 파일을 바이너리로 읽기
        base_64_encoded_data = base64.b64encode(binary_data)  # base64 인코딩
        base64_string = base_64_encoded_data.decode("utf-8")  # 문자열로 디코딩

    # 시스템 메시지: 모델의 역할 정의
    system = [
        {
            "text": "당신은 예술 분야의 전문가입니다. 사용자 요청에 대해서 대답해주세요."
        }
    ]

    # 사용자 메시지: 이미지와 텍스트 프롬프트를 함께 전달
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "image": {
                        "format": "jpg",  # 이미지 포맷
                        "source": {"bytes": base64_string},  # base64 인코딩된 이미지
                    }
                },
                {
                    "text": prompt  # 텍스트 프롬프트
                }
            ],
        }
    ]

    # 추론 파라미터 설정
    inference_params = {
        "maxTokens": 300,     # 최대 토큰 수
        "topP": 0.1,          # top-p 샘플링 (가장 확률 높은 토큰 누적합 기준)
        "topK": 20,           # top-k 샘플링 (확률 높은 상위 20개 중 선택)
        "temperature": 0.3    # 창의성 제어: 낮을수록 예측 가능
    }

    # 모델 요청 포맷 구성
    request_body = {
        "schemaVersion": "messages-v1",
        "messages": messages,
        "system": system,
        "inferenceConfig": inference_params,
    }

    # 모델 호출
    response = client.invoke_model(
        modelId=MODEL_ID,
        body=json.dumps(request_body)
    )

    # 응답 본문을 파싱
    model_response = json.loads(response["body"].read())

    # 사용량 정보 출력 (예: 사용한 토큰 수 등)
    print("\n[Usage]-------------------")
    print(json.dumps(model_response['usage'], indent=2))

    # 생성된 텍스트만 추출해서 출력
    content_text = model_response["output"]["message"]["content"][0]["text"]

    return content_text


In [2]:
# 실행
image_path = "sample/sunset.jpg"
prompt = "이미지에 대해서 상세하게 설명해주세요."
result = understand_image(image_path, prompt)
print("\n[Response Content Text]-------------------")
print(result)


[Usage]-------------------
{
  "inputTokens": 1839,
  "outputTokens": 283,
  "totalTokens": 2122,
  "cacheReadInputTokenCount": 0,
  "cacheWriteInputTokenCount": 0
}

[Response Content Text]-------------------
이 이미지는 해변의 일몰 풍경을 담고 있습니다. 넓은 바다와 낮은 해안선이 보이며, 바다 위에 떠 있는 작은 섬이 눈에 띕니다. 하늘은 구름이 많고, 일몰의 빛이 구름을 뚫고 바다 위로 퍼져 나가고 있습니다. 구름은 다양한 색조를 띠고 있어 일몰의 아름다움을 더욱 강조합니다. 바다 위에는 작은 파도가 보이며, 해안선 근처에는 몇 개의 바위가 흩어져 있습니다. 전체적으로 평온하고 고요한 분위기를 자아내는 풍경입니다.
